In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [4]:

test_data=pd.read_csv(r"C:\Users\MIS\Documents\HackerEarth\949ca43ea68511eb\dataset\test.csv")
train_data=pd.read_csv(r"C:\Users\MIS\Documents\HackerEarth\949ca43ea68511eb\dataset\train.csv")
test_data.head()

,tracking_id,datetime,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),...,area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),turbine_status,cloud_level,blade_length(m),blade_breadth(m),windmill_height(m)
0,WM_19817,2019-04-17 08:53:20,94.324266,17.641186,89.714193,51.146788,40.460560,39.594734,1073.202715,66.830037,...,24.004812,43.756693,445.976992,1664.222023,21.912243,BA,Medium,3.185837,0.403965,25.572431
1,WM_18723,2019-03-30 07:43:20,10.088870,13.978119,43.272846,46.516394,40.027788,41.176860,517.436430,37.284163,...,29.431813,42.728174,499.595287,1165.111992,-35.050093,A,Medium,3.016603,0.444755,24.371823
2,WM_34552,2019-08-10 11:33:20,347.152090,31.423035,41.076640,26.931602,43.109122,43.439556,1480.716492,70.010762,...,29.924235,43.256122,245.432231,1667.720491,27.195302,B2,Medium,2.611941,0.387368,27.654677
3,WM_28570,2019-06-26 03:53:20,24.471997,-99.000000,14.375078,66.513953,13.741253,15.577472,887.979475,41.445258,...,23.886434,13.501595,NaN,1329.744740,15.245757,BBB,Low,2.866805,0.450478,24.189426
4,WM_36934,2019-08-27 16:43:20,96.997026,33.281836,41.405192,1.843112,121.572907,43.934587,2053.916354,68.007787,...,35.906889,-99.000000,442.425744,691.408996,34.257024,A,Low,3.549672,0.368355,4.885440


In [7]:
#DATA PREPROCESSING

In [5]:
drop_columns = ['tracking_id','datetime']
submission = test_data[drop_columns]
train_data.drop(drop_columns,inplace=True,axis=1)
test_data.drop(drop_columns,inplace=True,axis=1)

In [6]:
train_data.isna().sum() / len(train_data) * 100

wind_speed(m/s)                    0.968085
atmospheric_temperature(°C)       12.234043
shaft_temperature(°C)              0.007092
blades_angle(°)                    0.765957
gearbox_temperature(°C)            0.003546
engine_temperature(°C)             0.042553
motor_torque(N-m)                  0.085106
generator_temperature(°C)          0.042553
atmospheric_pressure(Pascal)       9.599291
area_temperature(°C)               0.000000
windmill_body_temperature(°C)      8.379433
wind_direction(°)                 18.095745
resistance(ohm)                    0.003546
rotor_torque(N-m)                  2.028369
turbine_status                     6.237589
cloud_level                        0.978723
blade_length(m)                   18.060284
blade_breadth(m)                   0.000000
windmill_height(m)                 1.925532
windmill_generated_power(kW/h)     0.734043
dtype: float64

In [7]:
train_data.dtypes

wind_speed(m/s)                   float64
atmospheric_temperature(°C)       float64
shaft_temperature(°C)             float64
blades_angle(°)                   float64
gearbox_temperature(°C)           float64
engine_temperature(°C)            float64
motor_torque(N-m)                 float64
generator_temperature(°C)         float64
atmospheric_pressure(Pascal)      float64
area_temperature(°C)              float64
windmill_body_temperature(°C)     float64
wind_direction(°)                 float64
resistance(ohm)                   float64
rotor_torque(N-m)                 float64
turbine_status                     object
cloud_level                        object
blade_length(m)                   float64
blade_breadth(m)                  float64
windmill_height(m)                float64
windmill_generated_power(kW/h)    float64
dtype: object

In [8]:
train_data.loc[:,train_data.dtypes == 'object'].nunique()

turbine_status    14
cloud_level        3
dtype: int64

In [10]:
print(train_data['turbine_status'].value_counts(),'\n'*3,
train_data['cloud_level'].value_counts())

BB     1946
AAA    1939
BCB    1933
B2     1931
A      1930
D      1922
B      1882
AB     1868
ABC    1867
A2     1855
BA     1854
AC     1850
BD     1843
BBB    1821
Name: turbine_status, dtype: int64 


 Low              13921
Medium           13704
Extremely Low      299
Name: cloud_level, dtype: int64


In [13]:
continous_cols = train_data.dtypes[train_data.dtypes != 'object'].index
continous_cols_test = test_data.dtypes[test_data.dtypes != 'object'].index
(continous_cols)

Index(['wind_speed(m/s)', 'atmospheric_temperature(°C)',
       'shaft_temperature(°C)', 'blades_angle(°)', 'gearbox_temperature(°C)',
       'engine_temperature(°C)', 'motor_torque(N-m)',
       'generator_temperature(°C)', 'atmospheric_pressure(Pascal)',
       'area_temperature(°C)', 'windmill_body_temperature(°C)',
       'wind_direction(°)', 'resistance(ohm)', 'rotor_torque(N-m)',
       'blade_length(m)', 'blade_breadth(m)', 'windmill_height(m)',
       'windmill_generated_power(kW/h)'],
      dtype='object')

In [14]:
for col in continous_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mean())
for col in continous_cols_test:
    test_data[col] = test_data[col].fillna(test_data[col].mean())

In [15]:
train_data['cloud_level'].value_counts()
train_data['cloud_level'].fillna('Low',inplace = True)

In [16]:
train_data['turbine_status'].value_counts()
train_data['turbine_status'].fillna('BB',inplace = True)

In [17]:
test_data['cloud_level'].value_counts()
test_data['cloud_level'].fillna('Low',inplace = True)

In [18]:
test_data['turbine_status'].value_counts()
test_data['turbine_status'].fillna('BB',inplace = True)

In [19]:
test_data.isna().sum() / len(test_data) * 100

wind_speed(m/s)                  0.0
atmospheric_temperature(°C)      0.0
shaft_temperature(°C)            0.0
blades_angle(°)                  0.0
gearbox_temperature(°C)          0.0
engine_temperature(°C)           0.0
motor_torque(N-m)                0.0
generator_temperature(°C)        0.0
atmospheric_pressure(Pascal)     0.0
area_temperature(°C)             0.0
windmill_body_temperature(°C)    0.0
wind_direction(°)                0.0
resistance(ohm)                  0.0
rotor_torque(N-m)                0.0
turbine_status                   0.0
cloud_level                      0.0
blade_length(m)                  0.0
blade_breadth(m)                 0.0
windmill_height(m)               0.0
dtype: float64

In [20]:
train_data.describe()

,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
count,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,2.820000e+04,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000,28200.000000
mean,69.037071,0.383727,40.085387,-9.654038,41.027755,42.614239,1710.819803,65.027857,5.318506e+04,32.735091,20.799761,306.888883,1575.560011,25.849894,2.254034,0.397249,25.887052,6.130529
std,75.905527,41.481572,27.203465,47.734286,43.662831,6.123243,826.853447,19.812283,1.782768e+05,7.703391,52.029149,121.321415,483.317824,32.093408,10.206698,0.061158,7.698401,2.687601
min,-496.211029,-99.000000,-99.000000,-146.259543,-244.974098,3.167151,500.000000,33.893779,-1.188624e+06,-30.000000,-999.000000,0.000000,-1005.222988,-136.732217,-99.000000,0.200111,-30.295253,0.962305
25%,21.250907,0.383727,41.632797,-1.206613,40.557993,41.911899,870.621152,41.200625,1.689269e+04,27.311644,20.799761,246.793261,1268.141402,13.829618,2.254034,0.347445,24.469380,4.076167
50%,93.256368,14.150602,43.685853,-0.527911,43.221521,43.524018,2031.299986,70.722083,1.875303e+04,32.605195,42.495191,290.113549,1678.231923,31.549128,3.049007,0.398591,25.899163,5.791403
75%,95.248020,22.575311,45.673234,5.370015,45.879212,45.173397,2462.164052,78.939478,1.164252e+05,38.232387,44.325305,332.325695,1829.044723,41.337104,4.156647,0.449354,27.451080,7.931295
max,601.455670,80.217444,169.820455,165.932123,999.000000,50.000000,3000.000000,100.000000,1.272552e+06,55.000000,323.000000,569.966479,4693.481933,236.883264,18.209800,0.499975,78.351335,20.175358


In [21]:
test_data.describe()

,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m)
count,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,1.208600e+04,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000
mean,69.381862,0.687445,39.936571,-9.218509,40.318639,42.604738,1717.343991,65.237894,5.428332e+04,32.796168,20.906469,308.376140,1581.289398,26.566581,2.201328,0.397527,25.963743
std,75.721631,41.368648,27.113378,47.365263,42.480064,6.213838,829.431463,19.864025,1.781158e+05,7.671602,51.208956,121.366283,483.125121,31.728393,10.457889,0.061661,7.634222
min,-525.530600,-99.000000,-99.000000,-99.000000,-185.860883,-5.000000,500.000000,30.000000,-1.207259e+06,-30.000000,-99.000000,0.000000,-1654.449587,-182.169613,-99.000000,0.200009,-21.215639
25%,22.168108,0.687445,41.635178,-1.200378,40.459387,41.926556,874.600120,41.285062,1.690488e+04,27.407281,20.906469,247.242960,1272.310773,13.886059,2.201328,0.346909,24.490326
50%,93.288643,14.268344,43.707705,-0.529922,43.219696,43.551829,2031.691733,70.793474,1.870774e+04,32.659540,42.530522,291.206974,1680.225602,31.430853,3.044582,0.399637,25.963743
75%,95.228791,22.744798,45.755861,6.028020,45.840069,45.227942,2479.582123,79.427893,1.164036e+05,38.302580,44.355803,335.347247,1837.302026,41.759021,4.156727,0.450488,27.458013
max,525.434211,80.442232,183.299983,170.047823,563.000000,49.858945,2997.013769,99.571599,1.127571e+06,53.927698,191.979747,570.857751,4106.196001,213.260076,17.847936,0.499985,77.604969


In [22]:
#STANDARDIZING DATA

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
for col in continous_cols:
    train_data[col] = sc.fit_transform(train_data[col].values.reshape(-1,1))
    if(col == 'windmill_generated_power(kW/h)'):
        continue
    test_data[col] = sc.transform(test_data[col].values.reshape(-1,1))

In [24]:
train_data.describe()

,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
count,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04,2.820000e+04
mean,1.194887e-16,-5.041141e-16,-9.773112e-17,2.062357e-16,6.802626e-17,-1.111164e-15,-8.860131e-18,-4.844313e-16,1.904702e-17,-5.570327e-16,-1.446774e-16,2.744972e-16,-1.498024e-16,5.045609e-17,-5.347790e-17,6.587815e-17,1.105715e-16,-7.586524e-17
std,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00,1.000018e+00
min,-7.446864e+00,-2.395895e+00,-5.112873e+00,-2.861841e+00,-6.550352e+00,-6.442304e+00,-1.464396e+00,-1.571481e+00,-6.965749e+00,-8.143972e+00,-1.960089e+01,-2.529597e+00,-5.339817e+00,-5.065993e+00,-9.920527e+00,-3.223457e+00,-7.298048e+00,-1.923022e+00
25%,-6.295590e-01,4.550003e-17,5.688385e-02,1.769708e-01,-1.075903e-02,-1.147026e-01,-1.016158e+00,-1.202671e+00,-2.035768e-01,-7.040462e-01,1.638824e-15,-4.953510e-01,-6.360702e-01,-3.745470e-01,5.221243e-16,-8.143542e-01,-1.841548e-01,-7.643988e-01
50%,3.190772e-01,3.318852e-01,1.323556e-01,1.911894e-01,5.024422e-02,1.485807e-01,3.875969e-01,2.874140e-01,-1.931415e-01,-1.686248e-02,4.169935e-01,-1.382743e-01,2.124352e-01,1.775858e-01,7.788877e-02,2.194866e-02,1.573213e-03,-1.261838e-01
75%,3.453163e-01,5.349840e-01,2.054130e-01,3.147490e-01,1.111138e-01,4.179492e-01,9.086950e-01,7.021840e-01,3.547363e-01,7.136329e-01,4.521689e-01,2.096684e-01,5.244773e-01,4.825753e-01,1.864115e-01,8.519969e-01,2.031663e-01,6.700390e-01
max,7.014351e+00,1.924593e+00,4.769149e+00,3.678473e+00,2.194061e+01,1.206206e+00,1.559168e+00,1.765206e+00,6.839862e+00,2.890325e+00,5.808390e+00,2.168473e+00,6.451194e+00,6.575715e+00,1.563292e+00,1.679712e+00,6.815079e+00,5.225878e+00


In [25]:
test_data.describe()

,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m)
count,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000,12086.000000
mean,0.004542,0.007322,-0.005471,0.009124,-0.016241,-0.001552,0.007891,0.010602,0.006161,0.007929,0.002051,0.012259,0.011854,0.022332,-0.005164,0.004548,0.009962
std,0.997595,0.997295,0.996706,0.992287,0.972929,1.014813,1.003136,1.002629,0.999115,0.995891,0.984253,1.000388,0.999619,0.988644,1.024629,1.008243,0.991681
min,-7.833135,-2.395895,-5.112873,-1.871769,-5.196472,-7.776122,-1.464396,-1.768018,-7.070277,-8.143972,-2.302592,-2.529597,-6.683111,-6.481804,-9.920527,-3.225129,-6.118612
25%,-0.617475,0.007322,0.056971,0.177101,-0.013017,-0.112309,-1.011345,-1.198409,-0.203508,-0.691631,0.002051,-0.491644,-0.627443,-0.372788,-0.005164,-0.823117,-0.181434
50%,0.319502,0.334724,0.133159,0.191147,0.050202,0.153123,0.388071,0.291017,-0.193396,-0.009808,0.417673,-0.129261,0.216560,0.173900,0.077455,0.039050,0.009962
75%,0.345063,0.539070,0.208450,0.328534,0.110217,0.426857,0.929761,0.726837,0.354615,0.722745,0.452755,0.234574,0.541562,0.495722,0.186419,0.870534,0.204067
max,6.012806,1.930012,5.264665,3.764695,11.954825,1.183170,1.555556,1.743583,6.026614,2.751124,3.290137,2.175820,5.236059,5.839626,1.527838,1.679878,6.718127


In [26]:
label_enc = LabelEncoder()
train_data['cloud_level'] = label_enc.fit_transform(train_data['cloud_level'])
train_data['cloud_level'].value_counts()
test_data['cloud_level'] = label_enc.fit_transform(test_data['cloud_level'])
test_data['cloud_level'].value_counts()

1    6057
2    5893
0     136
Name: cloud_level, dtype: int64

In [27]:
train_data['cloud_level'] = train_data['cloud_level'].apply(lambda x:  x + 1)
test_data['cloud_level'] = test_data['cloud_level'].apply(lambda x:  x + 1)
train_data['cloud_level']
train_data['cloud_level'].value_counts()

2    14197
3    13704
1      299
Name: cloud_level, dtype: int64

In [28]:
enc = pd.get_dummies(train_data['turbine_status'])
train_data.drop('turbine_status',axis = 1,inplace = True)
train_data = train_data.join(enc)

In [29]:
enc = pd.get_dummies(test_data['turbine_status'])
test_data.drop('turbine_status',axis = 1,inplace = True)
test_data = test_data.join(enc)

In [30]:
#PREDICTIVE MODEL

In [31]:
x = train_data.drop(['windmill_generated_power(kW/h)'],axis = 1)
y = train_data['windmill_generated_power(kW/h)']

In [33]:
from sklearn.model_selection import cross_val_score,ShuffleSplit,GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor,StackingRegressor,AdaBoostRegressor,GradientBoostingRegressor,BaggingRegressor,VotingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

ImportError: cannot import name 'StackingRegressor' from 'sklearn.ensemble' (C:\Users\MIS\Anaconda3\lib\site-packages\sklearn\ensemble\__init__.py)